In [1]:
import sys
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
import re
import math
from pyspark.sql.window import Window
from pyspark.sql import SparkSession, functions, types
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor,DecisionTreeRegressor,GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
from pyspark.ml.feature import MinMaxScaler
import pandas as pd
spark = SparkSession.builder.appName('example code').getOrCreate()
assert spark.version >= '2.4' # make sure we have Spark 2.4+
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext
spark.conf.set('spark.sql.session.timeZone', 'UTC')

bitcoin_schema = types.StructType([
    types.StructField('Date', types.StringType()),
    types.StructField('Symbol', types.StringType()),
    types.StructField('Open', types.FloatType()),
    types.StructField('High', types.FloatType()),
    types.StructField('Low', types.FloatType()),
    types.StructField('Close', types.FloatType()),
    types.StructField('Volume BTC', types.FloatType()),
    types.StructField('Volume USD', types.FloatType()),
])
tweets_schema = types.StructType([
    types.StructField('Date', types.StringType()),
    types.StructField('tweet', types.StringType())
])

In [2]:
col_order = ["timestamp", "Open", "High", "Low", "Close", "Volume BTC", "Volume USD"]

In [3]:
bitcoin_df = spark.read.csv("bitcoin-usd-hour.csv", schema=bitcoin_schema).cache()
bitcoin_df = bitcoin_df.withColumn("timestamp", functions.unix_timestamp("Date", 'yyyy-MM-dd hh-aa').cast("timestamp")).drop('Date')
bitcoin_df = bitcoin_df.select(col_order).sort("timestamp").cache()
bitcoin_df.show(3)

+-------------------+-------+-------+-------+-------+----------+----------+
|          timestamp|   Open|   High|    Low|  Close|Volume BTC|Volume USD|
+-------------------+-------+-------+-------+-------+----------+----------+
|2017-07-01 11:00:00| 2562.3|2580.99| 2554.0| 2568.0|      3.04|   7781.15|
|2017-07-01 12:00:00| 2568.0|2577.79|2555.01|2576.58|      11.0|  28231.42|
|2017-07-01 13:00:00|2576.58|2576.58|2555.01|2555.08|     14.23|  36482.36|
+-------------------+-------+-------+-------+-------+----------+----------+
only showing top 3 rows



In [4]:
tweet_df = spark.read.csv("tweets_history.csv", schema=tweets_schema)
tweet_df = tweet_df.withColumn("timestamp", functions.unix_timestamp("Date", 'yyyy-MM-dd HH:mm:ss').cast("timestamp")).drop('Date')
tweet_df = tweet_df.select("timestamp", "tweet").sort("timestamp").cache()
tweet_df = tweet_df.na.drop()

tweet_df.show(3)

+-------------------+--------------------+
|          timestamp|               tweet|
+-------------------+--------------------+
|2019-11-17 00:00:01|b'Bitcoin - BTC\n...|
|2019-11-17 00:00:01|b'#HourlyCryptoSt...|
|2019-11-17 00:00:02|"b""Join 'The Sig...|
+-------------------+--------------------+
only showing top 3 rows



In [5]:
import preprocessor as p
def function_udf(input_str):
    input_str = re.sub(r'b', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = tweet_df.withColumn('CleanedTweets', func_udf(tweet_df['tweet']))
CleanDF.show(3)

NameError: name 'udf' is not defined